In [2]:
!pip install langchain
from langchain.text_splitter import SpacyTextSplitter
with open('test.txt') as f:
  data = f.read()

text_splitter = SpacyTextSplitter(chunk_size=500)
texts = text_splitter.split_text(data)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [3]:
import pandas as pd
df = pd.DataFrame({'facts': texts})
df.head()

,facts
0,Bismarck was the first of two Bismarck-class b...
1,Bismarck and her sister ship Tirpitz were the ...
2,"The ship, along with the heavy cruiser Prinz E..."
3,"At the Battle of the Denmark Strait, the battl..."
4,Bismarck suffered sufficient damage from three...


In [5]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
text = df['facts']
encoder = SentenceTransformer('paraphrase-mpnet-base-v2')
vectors = encoder.encode(texts)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
!pip install faiss-cpu
import faiss
vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [11]:
import numpy as np
def search(search_text):
  search_vector = encoder.encode(search_text)
  _vector = np.array([search_vector])
  faiss.normalize_L2(_vector)
  k=index.ntotal
  distances, ann = index.search(_vector, k=k)
  results = pd.DataFrame({'distances':distances[0], 'ann': ann[0]})
  merge = pd.merge(results, df, left_on='ann', right_index=True)
  return merge


In [16]:
merge = search('When was wreck of Bismarck discovered?')
for i in range(0,5):
  print(merge['facts'][i])
  print('-------------------------------------------------')



The next day the German trawler Sachsenwald rescued another two from a raft at 22:45.[170]

One of the men picked up by the British died of his wounds the following day.

Out of a crew of over 2,200 men, only 114 survived.[167]

Wreckage
Discovery by Robert Ballard

Painting by Ken Marschall depicting the unmanned probe Argo exploring the wreck
The wreck of Bismarck was discovered on 8 June 1989 by Robert Ballard, the oceanographer responsible for finding RMS Titanic.[171]
-------------------------------------------------
While no one onboard Bismarck when it sunk could confirm or deny that, Prinz Eugen's stern collapsed after it was similarly torpedoed in the stern in 1942.

This prompted the Germans to straighten the stern structures on all German capital ships.[180]

Subsequent expeditions
In June 2001, Deep Ocean Expeditions, partnered with Woods Hole Oceanographic Institution, conducted another investigation of the wreck.

The researchers used Russian-built mini-submarines.
------